In [1]:
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import sys
from divtel.telescope import Telescope, Array
#from divtel. import *
from astropy.coordinates import SkyCoord
from divtel import pointing
import healpy as hp
import tqdm

In [7]:
def hess_1():
    tel1 = Telescope(100*u.m, 0*u.m, 0*u.m, 15*u.m, 0.65*u.m)
    tel2 = Telescope(0 * u.m, 100 * u.m, 0 * u.m, 15 * u.m, 0.65 * u.m)
    tel3 = Telescope(-100 * u.m, 0 * u.m, 0 * u.m, 15 * u.m, 0.65 * u.m)
    tel4 = Telescope(0 * u.m, -100 * u.m, 0 * u.m, 15 * u.m, 0.65 * u.m)
    return Array([tel1, tel2, tel3, tel4])

def hess_2():
    tel1 = Telescope(100*u.m, 0*u.m, 0*u.m, 15*u.m, 0.65 * u.m)
    tel2 = Telescope(0 * u.m, 100 * u.m, 0 * u.m, 15 * u.m, 0.65 * u.m)
    tel3 = Telescope(-100 * u.m, 0 * u.m, 0 * u.m, 15 * u.m, 0.65 * u.m)
    tel4 = Telescope(0 * u.m, -100 * u.m, 0 * u.m, 15 * u.m, 0.65 * u.m)
    tel5 = Telescope(0 * u.m, 0. * u.m, 0 * u.m, 36 * u.m, 1. * u.m)
    return Array([tel1, tel2, tel3, tel4, tel5])

def hess_16():
    list_tel = []
    for x in np.linspace(-187.5, 187.5, 4):
        for y in np.linspace(-187.5, 187.5, 4):
            list_tel.append(Telescope(x*u.m, y*u.m, 0*u.m, 15*u.m, 0.65 * u.m))
    return Array([list_tel])

def hess_64():
    list_tel = []
    for x in np.linspace(-1000, 1000, 8):
        for y in np.linspace(-1000, 1000, 8):
            
            list_tel.append(Telescope(x*u.m, y*u.m, 0*u.m, 15*u.m, 0.65 * u.m))
    return Array(list_tel)



def random_array(n=10):
    tels = [Telescope(1000*np.random.rand()*u.m,
                      1000*np.random.rand() * u.m,
                      0 * u.m,
                      np.random.rand() * u.m,
                      np.random.rand() * u.m,
                      )
            for i in range(n)
            ]
    array = Array(tels)
    for tel in array.telescopes:
        tel.x -= array.barycenter[0]*u.m
        tel.y -= array.barycenter[0]*u.m
        tel.z -= array.barycenter[0]*u.m
    return Array(tels)

In [13]:
def sst_64():
    list_tel = []
    for x in np.linspace(-1000, 1000, 8):
        for y in np.linspace(-1000, 1000, 8):
            list_tel.append(Telescope(x*u.m, y*u.m, 0*u.m, 2.15*u.m, 0.2 * u.m))
    return Array(list_tel)

In [14]:
array = sst_64()

for tel in array.telescopes:
    tel.point_to_altaz(90*u.deg, 0*u.deg)

In [15]:
#array = LoadConfig("../config/Paranal_prod6_alpha4LSTs_reduced_list.txt")
div=0.01
alt=70*u.deg
az=0*u.deg
array.divergent_pointing(0.01, alt_mean=70*u.deg, az_mean=0*u.deg)

In [16]:
nside = 512
map_multiplicity = np.zeros(hp.nside2npix(nside), dtype=np.int8)
counter = np.arange(0, hp.nside2npix(nside))
ra, dec = hp.pix2ang(nside, counter, True, lonlat=True)
coordinate = SkyCoord(ra=ra*u.deg, dec=dec*u.deg)
for i in tqdm.tqdm(range(len(array.telescopes))):
    pointing = SkyCoord(ra=telescope.point_to_altaz[i,1], dec=array.point_to_altaz[i,0])
    r_fov = np.arctan((array.telescopes[i].camera_radius/array.telescopes[i].focal).to(u.dimensionless_unscaled)).to(u.deg)
    mask = coordinate.separation(pointing) < r_fov
    map_multiplicity[mask] += 1
print(hp.nside2npix(nside), hp.nside2pixarea(nside, True), hp.nside2resol(nside, True))
mask_fov = map_multiplicity>0
mask_fov_eff = map_multiplicity>2
print(hp.nside2pixarea(nside, True)*np.sum(mask_fov),
      hp.nside2pixarea(nside, True)*np.sum(mask_fov_eff),
      np.sum(mask_fov_eff)/np.sum(mask_fov))
print(np.mean(map_multiplicity[mask_fov]),
      np.mean(map_multiplicity[mask_fov_eff]))

  0%|                                                    | 0/64 [00:00<?, ?it/s]


NameError: name 'telescope' is not defined

In [ ]:
hp.cartview(map_multiplicity, rot=[az, alt], lonra=[-25,25], latra=[-25,25], nest=True)
hp.graticule()

In [ ]:
hp.cartview(map_multiplicity>3, rot=[az, alt], lonra=[-25,25], latra=[-25,25], nest=True)
hp.graticule()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15,4))
ax = array.display_2d(projection='xz', ax=axes[0])
array.display_2d(projection='xy', ax=axes[1])
array.display_2d(projection='yz', ax=axes[2])

In [ ]:
array.divergent_pointing(div, alt*u.deg, az*u.deg)
ax = array.display_3d()
ax.scatter(array.barycenter[0], array.barycenter[1], array.barycenter[2])

In [ ]:
array.pointing_altaz.shape

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.scatter(array.pointing_altaz[:,1].to_value(u.rad), 90.*u.deg-array.pointing_altaz[:,0].to(u.deg))
ax.set_rmax(90)
ax.set_rmin(0)
ax.grid(True)

ax.set_title("A line plot on a polar axis", va='bottom')
plt.show()

In [ ]:
plt.scatter(array.pointing_altaz[:,1].to_value(u.deg)*np.sin(90.*u.deg-array.pointing_altaz[:,0].to(u.deg)),
            90.*u.deg-array.pointing_altaz[:,0].to(u.deg))